In [183]:
example = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [184]:
translate = ["destination", "source", "length"]
example_parsed_data = {}
for row in example.splitlines():
    if row == "":
        continue
    if ":" in row:
        key, value = row.split(":")
        if "seeds" in key:
            example_parsed_data[key.strip()] = [int(x) for x in value.split()]
        else:
            example_parsed_data[key.strip()] = []
    else:
        temp = {translate[i]: int(x) for i, x in enumerate(row.split())}
        ranges = {
            "source": range(temp["source"], temp["source"] + temp["length"]),
            "destination": range(
                temp["destination"], temp["destination"] + temp["length"]
            ),
        }
        example_parsed_data[key.strip()].append(ranges)

for i in example_parsed_data.keys():
    if "seeds" not in i:
        example_parsed_data[i] = sorted(
            example_parsed_data[i], key=lambda x: x["source"].start
        )

In [185]:
def find_destination(source_number, translate_dir, step):
    specific_translate = translate_dir[step]
    for translate in specific_translate:
        if source_number in translate["source"]:
            return translate["destination"].start + (
                source_number - translate["source"].start
            )
    return source_number

# Part 1

In [186]:
with open("./data/Day 5/input.txt", 'r') as file:
    data = file.read().splitlines()

In [187]:
translate = ["destination", "source", "length"]
parsed_data = {}
for row in data:
    if row == "":
        continue
    if ":" in row:
        key, value = row.split(":")
        if "seeds" in key:
            parsed_data[key.strip()] = [int(x) for x in value.split()]
        else:
            parsed_data[key.strip()] = []
    else:
        temp = {translate[i]: int(x) for i, x in enumerate(row.split())}
        ranges = {
            "source": range(temp["source"], temp["source"] + temp["length"]),
            "destination": range(
                temp["destination"], temp["destination"] + temp["length"]
            ),
        }
        parsed_data[key.strip()].append(ranges)

for i in parsed_data.keys():
    if "seeds" not in i:
        parsed_data[i] = sorted(parsed_data[i], key=lambda x: x["source"].start)

In [188]:
locations = []
for seed in example_parsed_data["seeds"]:
    soil = find_destination(seed, example_parsed_data, "seed-to-soil map")
    fertilizer = find_destination(soil, example_parsed_data, "soil-to-fertilizer map")
    water = find_destination(fertilizer, example_parsed_data, "fertilizer-to-water map")
    light = find_destination(water, example_parsed_data, "water-to-light map")
    temperature = find_destination(
        light, example_parsed_data, "light-to-temperature map"
    )
    humidity = find_destination(
        temperature, example_parsed_data, "temperature-to-humidity map"
    )
    location = find_destination(
        humidity, example_parsed_data, "humidity-to-location map"
    )
    locations.append(location)
print(min(locations))

35


In [189]:
locations = []
for seed in parsed_data["seeds"]:
    soil = find_destination(seed, parsed_data, "seed-to-soil map")
    fertilizer = find_destination(soil, parsed_data, "soil-to-fertilizer map")
    water = find_destination(fertilizer, parsed_data, "fertilizer-to-water map")
    light = find_destination(water, parsed_data, "water-to-light map")
    temperature = find_destination(light, parsed_data, "light-to-temperature map")
    humidity = find_destination(temperature, parsed_data, "temperature-to-humidity map")
    location = find_destination(humidity, parsed_data, "humidity-to-location map")
    locations.append(location)
print(min(locations))

218513636


## Part 2

In [190]:
seeds = example_parsed_data["seeds"]
new_seeds = []
for i in range(0, len(seeds), 2):
    new_seeds.append(range(seeds[i], seeds[i + 1] + seeds[i] + 1))
example_parsed_data["seedranges"] = new_seeds


seeds = parsed_data["seeds"]
new_seeds = []
for i in range(0, len(seeds), 2):
    new_seeds.append(range(seeds[i], seeds[i + 1] + seeds[i] + 1))
parsed_data["seedranges"] = new_seeds

In [202]:
def map_range(input_range, range_info, key):
    """The function should find the overlap between the input range and the range list.
    Example input: input_range = range(52, 100), range_list = [range(40, 60), range(70, 90)]
    Example source_output: [range(52, 60), range(60,70), range(70, 90), range(90, 100)]
    """
    range_source_list = [x["source"] for x in range_info[key]]
    range_destination_list = [x["destination"] for x in range_info[key]]
    source_output = []
    destination_output = []
    for nr, i in enumerate(range_source_list):
        if i.start >= input_range.stop or i.stop <= input_range.start:
            continue
        elif i.start >= input_range.start and i.stop <= input_range.stop:
            source_output.append(range(i.start, i.stop))
            destination_output.append(
                range(range_destination_list[nr].start, range_destination_list[nr].stop)
            )
        elif i.start < input_range.start and i.stop > input_range.stop:
            source_output.append(range(input_range.start, input_range.stop))
            destination_output.append(
                range(
                    range_destination_list[nr].start + (input_range.start - i.start),
                    range_destination_list[nr].start + (input_range.stop - i.start),
                )
            )
        elif i.start < input_range.start and i.stop <= input_range.stop:
            source_output.append(range(input_range.start, i.stop))
            destination_output.append(
                range(
                    range_destination_list[nr].start + (input_range.start - i.start),
                    range_destination_list[nr].stop,
                )
            )
        elif i.start >= input_range.start and i.stop >= input_range.stop:
            source_output.append(range(i.start, input_range.stop))
            destination_output.append(
                range(
                    range_destination_list[nr].start,
                    range_destination_list[nr].start + (input_range.stop - i.start),
                )
            )

    source_output = sorted(source_output, key=lambda x: x.start)

    # Add stuff which isn't mapped in the range list
    if source_output:
        if source_output[0].start > input_range.start:
            destination_output.append(range(input_range.start, source_output[0].start))
            source_output.insert(0, range(input_range.start, source_output[0].start))
        if source_output[-1].stop < input_range.stop:
            destination_output.append(range(source_output[-1].stop, input_range.stop))
            source_output.append(range(source_output[-1].stop, input_range.stop))

        for i in range(len(source_output) - 1):
            if source_output[i].stop != source_output[i + 1].start:
                destination_output.append(
                    range(source_output[i].stop, source_output[i + 1].start)
                )
                source_output.insert(
                    i + 1, range(source_output[i].stop, source_output[i + 1].start)
                )
    else:
        source_output.append(input_range)
        destination_output.append(input_range)
    destination_output = sorted(destination_output, key=lambda x: x.start)
    return source_output, destination_output


map_range(range(95, 110), example_parsed_data, "seed-to-soil map")

([range(95, 98), range(98, 100), range(100, 110)],
 [range(50, 52), range(97, 100), range(100, 110)])

In [205]:
seedranges = example_parsed_data["seedranges"]
locations = []
start_locs = []
for seedrange in seedranges:
    _, soil_range = map_range(seedrange, example_parsed_data, "seed-to-soil map")
    for soil in soil_range:
        _, fertilizer_range = map_range(
            soil, example_parsed_data, "soil-to-fertilizer map"
        )
        for fertilizer in fertilizer_range:
            _, water_range = map_range(
                fertilizer, example_parsed_data, "fertilizer-to-water map"
            )
            for water in water_range:
                _, light_range = map_range(
                    water, example_parsed_data, "water-to-light map"
                )
                for light in light_range:
                    _, temperature_range = map_range(
                        light, example_parsed_data, "light-to-temperature map"
                    )
                    for temperature in temperature_range:
                        _, humidity_range = map_range(
                            temperature,
                            example_parsed_data,
                            "temperature-to-humidity map",
                        )
                        for humidity in humidity_range:
                            _, location_range = map_range(
                                humidity,
                                example_parsed_data,
                                "humidity-to-location map",
                            )
                            locations.append(location_range)
                            start_locs.append(min([x.start for x in location_range]))
print(min(start_locs))

46


In [206]:
seedranges = parsed_data["seedranges"]
locations = []
start_locs = []
for seedrange in seedranges:
    _, soil_range = map_range(seedrange, parsed_data, "seed-to-soil map")
    for soil in soil_range:
        _, fertilizer_range = map_range(soil, parsed_data, "soil-to-fertilizer map")
        for fertilizer in fertilizer_range:
            _, water_range = map_range(
                fertilizer, parsed_data, "fertilizer-to-water map"
            )
            for water in water_range:
                _, light_range = map_range(water, parsed_data, "water-to-light map")
                for light in light_range:
                    _, temperature_range = map_range(
                        light, parsed_data, "light-to-temperature map"
                    )
                    for temperature in temperature_range:
                        _, humidity_range = map_range(
                            temperature,
                            parsed_data,
                            "temperature-to-humidity map",
                        )
                        for humidity in humidity_range:
                            _, location_range = map_range(
                                humidity,
                                parsed_data,
                                "humidity-to-location map",
                            )
                            locations.append(location_range)
                            start_locs.append(min([x.start for x in location_range]))
print(min(start_locs))

81956384
